# IBM Applied Data Science Capstone Course by Coursera
## Week 5 Final Report
### Opening a New Shopping Mall in Vishakapatnam,INDIA 

* Build a dataframe of neighborhoods in Visakhapatnam, INDIA by web scraping the data from Wikipedia page
+ Get the geographical coordinates of the neighborhoods
+ Obtain the venue data for the neighborhoods from Foursquare API
+ Explore and cluster the neighborhoods
+ Select the best cluster to open a new shopping mall

### 1. Import libraries

In [47]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import json # library to handle JSON files
from urllib.request import urlopen
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values# to get coordinates
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

# All requested packages already installed.

Folium installed and imported!


### 2. Scrap data from Wikipedia page into a DataFrame

In [48]:
url="https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Visakhapatnam"
html= urlopen(url)

In [49]:
soup=BeautifulSoup(html.read(),'lxml')
type(soup)

bs4.BeautifulSoup

In [50]:
alllinks=soup.find_all('a')
lst=[]
for title in alllinks:
    lst.append(title.get_text())

In [51]:
lst
str_test=str(lst)
str_test

"['', 'Help', 'Jump to navigation', 'Jump to search', 'category', 'Visakhapatnam', '', 'Suburbs of Visakhapatnam', 'learn more', 'Abidnagar', 'Adarsh Nagar', 'Adavivaram', 'Aganampudi', 'Akkayyapalem', 'Akkireddypalem', 'Allipuram', 'Anakapalle', 'Anandapuram', 'Appikonda', 'Arilova', 'Asilmetta', 'Atchutapuram', 'Balayya Sastri Layout', 'Beach Road, Visakhapatnam', 'Bheemunipatnam', 'CBM Compound', 'Chengal Rao Peta', 'Chinagantyada', 'Chinna Gadhili', 'Chinna Waltair', 'Chinnamushidiwada', 'Chintalagraharam', 'Daba Gardens', 'Daspalla Hills', 'Desapatrunipalem', 'Devada', 'Devipuram', 'Dondaparthy', 'Duvvada', 'Dwaraka Nagar', 'Gajuwaka', 'Gambhiram', 'Gandhigram, Visakhapatnam', 'Gangavaram, Visakhapatnam', 'Gidijala', 'Gnanapuram', 'Gopalapatnam', 'Gudilova', 'Hanumanthavaka', 'HB Colony', 'Isukathota', 'Jagadamba Centre', 'Jalari Peta', 'Jodugullapalem', 'Kailasapuram, Visakhapatnam', 'Kakani Nagar', 'Kancharapalem', 'Kapuluppada', 'Kirlampudi Layout', 'Kommadi', 'Kurmannapalem', 

In [52]:
neighbor= pd.DataFrame(lst,columns=['Neighbors'])

In [53]:
new=neighbor.drop(neighbor.index[:9])
neigh=new.drop(new.index[121:])
final=neigh.reset_index(drop=True)
df=pd.DataFrame(final)

In [54]:
df.shape

(121, 1)

### 3. Get the geographical coordinates

In [55]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7e845aa89744488aab81d912255e398d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6yTEedEQ3k75kfeg7USkyuf_382UQoP-bwOfHVcp7Buq',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_7e845aa89744488aab81d912255e398d.get_object(Bucket='rizwan-donotdelete-pr-r3zbdr5itlvnbp',Key='rio (1).xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_coor = pd.read_excel(body,index_col=0)
df_coor.head()

,Neighbors,Latitude,Longitude
0,"Abidnagar, Visakhapatnam",17.738865,83.299000
1,"Atchutapuram, Visakhapatnam",17.564214,82.978877
2,"Adarsh Nagar, Visakhapatnam",17.763627,83.332847
3,"Adavivaram, Visakhapatnam",17.776142,83.246586
4,"Aganampudi, Visakhapatnam",17.686435,83.137766


In [56]:
df_coor.shape

(121, 3)

### 4. Create a map ofVisakhapatnam with neighborhoods superimposed on top

In [57]:
map_vizag = folium.Map(location=[17.6868, 83.2185], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Neighbors']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_vizag)  
    
map_vizag

### 5. Use the Foursquare API to explore the neighborhoods

In [58]:
# define Foursquare Credentials and Version
CLIENT_ID = 'NTDWXJVZVZJRG4UPSSYPFMVSOLHHJWZ2VQRTSGKSCZOJOBBM' # your Foursquare ID
CLIENT_SECRET = 'WVKNOASUOZQ5H2HDCVFNWYZXHJWSQTRYHX1B3ZKO22D2PF12' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NTDWXJVZVZJRG4UPSSYPFMVSOLHHJWZ2VQRTSGKSCZOJOBBM
CLIENT_SECRET:WVKNOASUOZQ5H2HDCVFNWYZXHJWSQTRYHX1B3ZKO22D2PF12


<b>Now, let's get the top 100 venues that are within a radius of 3300 meters.</b>

In [59]:
import requests
radius = 3300
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_coor['Latitude'], df_coor['Longitude'], df_coor['Neighbors']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [60]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3475, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Abidnagar, Visakhapatnam",17.738865,83.299,Food Ex,17.722155,83.318422,Fast Food Restaurant
1,"Abidnagar, Visakhapatnam",17.738865,83.299,Sweet India,17.725087,83.316572,Snack Place
2,"Abidnagar, Visakhapatnam",17.738865,83.299,Cream & Fudge,17.719339,83.311927,Ice Cream Shop
3,"Abidnagar, Visakhapatnam",17.738865,83.299,Sai Ram Parlour,17.726339,83.303465,Indian Restaurant
4,"Abidnagar, Visakhapatnam",17.738865,83.299,Sangam Sarat Theatre,17.725508,83.302463,Indie Movie Theater


<b>Let's check how many venues were returned for each neighorhood</b>

In [61]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Abidnagar, Visakhapatnam",61,61,61,61,61,61
"Adarsh Nagar, Visakhapatnam",20,20,20,20,20,20
"Adavivaram, Visakhapatnam",3,3,3,3,3,3
"Aganampudi, Visakhapatnam",5,5,5,5,5,5
"Akkayyapalem, Visakhapatnam",42,42,42,42,42,42
"Akkireddypalem, Visakhapatnam",6,6,6,6,6,6
"Allipuram, Visakhapatnam",43,43,43,43,43,43
"Anakapalle, Visakhapatnam",6,6,6,6,6,6
"Anandapuram, Visakhapatnam",1,1,1,1,1,1


In [62]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 94 uniques categories.


<b>Let's find out how many unique categories can be curated from all the returned venues</b>

In [63]:
venues_df['VenueCategory'].unique()[:50]

array(['Fast Food Restaurant', 'Snack Place', 'Ice Cream Shop',
       'Indian Restaurant', 'Indie Movie Theater', 'Café', 'Restaurant',
       'Pizza Place', 'Fabric Shop', 'Bookstore', 'Breakfast Spot',
       'Multicuisine Indian Restaurant', 'Italian Restaurant', 'Hotel',
       'Coffee Shop', 'Multiplex', 'Shopping Mall', 'Dessert Shop',
       'Department Store', 'Train Station', 'Sandwich Place',
       'Food Court', 'Vegetarian / Vegan Restaurant', 'Park',
       'Cricket Ground', 'Volleyball Court', 'Bakery',
       'Convenience Store', 'Golf Course', 'Dhaba', 'ATM', 'Mountain',
       'Diner', 'Bus Station', 'Trail', 'Beach', 'Cafeteria',
       'Historic Site', 'Garden Center', 'Skating Rink', 'Steakhouse',
       'Clothing Store', 'Airport', 'Juice Bar', 'Harbor / Marina',
       'Pier', 'Pharmacy', 'Athletics & Sports', 'Campground',
       'Playground'], dtype=object)

In [64]:
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [65]:
vizag_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vizag_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [vizag_onehot.columns[-1]] + list(vizag_onehot.columns[:-1])
vizag_onehot = vizag_onehot[fixed_columns]

print(vizag_onehot.shape)
vizag_onehot.head()

(3475, 95)


,Neighborhoods,ATM,Accessories Store,Airport,American Restaurant,Andhra Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bar,...,Snack Place,Spa,Steakhouse,Supermarket,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court
0,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Abidnagar, Visakhapatnam",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [66]:
vizag_grouped = vizag_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(vizag_grouped.shape)
vizag_grouped.head()

(115, 95)


,Neighborhoods,ATM,Accessories Store,Airport,American Restaurant,Andhra Restaurant,Asian Restaurant,Athletics & Sports,Bakery,Bar,...,Snack Place,Spa,Steakhouse,Supermarket,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court
0,"Abidnagar, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,...,0.016393,0.0,0.00000,0.0,0.0,0.0,0.016393,0.016393,0.0,0.016393
1,"Adarsh Nagar, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.050000,0.0,0.00000,0.0,0.0,0.1,0.000000,0.000000,0.0,0.000000
2,"Adavivaram, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
3,"Aganampudi, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.600000,0.000000,0.0,0.000000
4,"Akkayyapalem, Visakhapatnam",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.023810,0.0,0.02381,0.0,0.0,0.0,0.023810,0.000000,0.0,0.023810


In [67]:
len(vizag_grouped[vizag_grouped["Shopping Mall"] > 0])

48

<b> Create a new DataFrame for Shopping Mall data only</b>

In [68]:
vizag_mall = vizag_grouped[["Neighborhoods","Shopping Mall"]]

In [69]:
vizag_mall.head()

,Neighborhoods,Shopping Mall
0,"Abidnagar, Visakhapatnam",0.04918
1,"Adarsh Nagar, Visakhapatnam",0.00000
2,"Adavivaram, Visakhapatnam",0.00000
3,"Aganampudi, Visakhapatnam",0.00000
4,"Akkayyapalem, Visakhapatnam",0.02381


### 7. Cluster Neighborhoods

In [70]:
# set number of clusters
kclusters = 3

vizag_clustering = vizag_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vizag_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 0, 1, 0, 1, 1, 1], dtype=int32)

In [71]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
vizag_merged = vizag_mall.copy()

# add clustering labels
vizag_merged["Cluster Labels"] = kmeans.labels_

In [72]:
vizag_merged.head()

,Neighborhoods,Shopping Mall,Cluster Labels
0,"Abidnagar, Visakhapatnam",0.04918,0
1,"Adarsh Nagar, Visakhapatnam",0.00000,1
2,"Adavivaram, Visakhapatnam",0.00000,1
3,"Aganampudi, Visakhapatnam",0.00000,1
4,"Akkayyapalem, Visakhapatnam",0.02381,0


In [73]:
# merge vizag_grouped with vizag_data to add latitude/longitude for each neighborhood
vizag_merged = vizag_merged.join(df_coor.set_index("Neighbors"), on="Neighborhoods")

print(vizag_merged.shape)
vizag_merged.head() # check the last columns!

(116, 5)


,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Abidnagar, Visakhapatnam",0.04918,0,17.738865,83.299000
1,"Adarsh Nagar, Visakhapatnam",0.00000,1,17.763627,83.332847
2,"Adavivaram, Visakhapatnam",0.00000,1,17.776142,83.246586
3,"Aganampudi, Visakhapatnam",0.00000,1,17.686435,83.137766
4,"Akkayyapalem, Visakhapatnam",0.02381,0,17.735277,83.299952


In [74]:
# sort the results by Cluster Labels
print(vizag_merged.shape)
vizag_merged.sort_values(["Cluster Labels"], inplace=True)
vizag_merged.head()

(116, 5)


,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Abidnagar, Visakhapatnam",0.049180,0,17.738865,83.299000
76,"Poorna Market, Visakhapatnam",0.028169,0,17.709025,83.297086
93,"Seethammapeta, Visakhapatnam",0.019231,0,17.731248,83.306905
23,"Daba Gardens, Visakhapatnam",0.023256,0,17.716031,83.297420
92,"Seethammadhara, Visakhapatnam",0.047619,0,17.742199,83.309109


<b>Finally, let's visualize the resulting clusters</b>

In [75]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[17.6868, 83.2185], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vizag_merged['Latitude'], vizag_merged['Longitude'], vizag_merged['Neighborhoods'], vizag_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 8. Examine Clusters

In [76]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<b>Cluster 0</b>

In [77]:
#cluster:0
vizag_merged.loc[vizag_merged['Cluster Labels'] == 0].head()

,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Abidnagar, Visakhapatnam",0.049180,0,17.738865,83.299000
76,"Poorna Market, Visakhapatnam",0.028169,0,17.709025,83.297086
93,"Seethammapeta, Visakhapatnam",0.019231,0,17.731248,83.306905
23,"Daba Gardens, Visakhapatnam",0.023256,0,17.716031,83.297420
92,"Seethammadhara, Visakhapatnam",0.047619,0,17.742199,83.309109


<b>Cluster 1</b>

In [78]:
#cluster:1
vizag_merged.loc[vizag_merged['Cluster Labels'] == 1].head()

,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
66,"Narava, Visakhapatnam",0.000000,1,17.744803,83.182360
67,"Nathayyapalem, Visakhapatnam",0.000000,1,17.711827,83.196381
68,"Nidigattu, Visakhapatnam",0.000000,1,17.861958,83.386952
69,"One Town, Visakhapatnam",0.000000,1,17.695621,83.293837
71,"Pandurangapuram, Visakhapatnam",0.016393,1,17.715473,83.321994


<b>Cluster 2</b>

In [79]:
#cluster:2
vizag_merged.loc[vizag_merged['Cluster Labels'] == 2]

,Neighborhoods,Shopping Mall,Cluster Labels,Latitude,Longitude
98,"Sriharipuram, Visakhapatnam",0.2,2,17.687423,83.235879


<b>Observations:</b>

Most of the shopping malls are concentrated in the central area of Visakhpatnam city, with the highest number in cluster 2 and moderate number in cluster 0. On the other hand, cluster 1 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.